In [26]:
import pickle
import matplotlib.pyplot as plt
import pandas as pd
import cv2
from PIL import Image
import numpy as np

import statsmodels.api as sm
import random
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from keras.callbacks import EarlyStopping
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import r2_score as rSquared
from keras.models import Sequential
from keras.layers import Conv2D, Dense

In [27]:
from google.colab import drive
drive.mount('/content/drive')

with open('/content/drive/MyDrive/FYP/CNN_trainset.pkl','rb') as f:
    train = pickle.load(f)

with open('/content/drive/MyDrive/FYP/CNN_testset.pkl','rb') as f:
    test = pickle.load(f)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


NameError: name 'images' is not defined

In [ ]:
train[0]

In [ ]:
image_size=224
X_train2 = np.array([item[0] for item in train]).reshape(-1,image_size,image_size,3)
X_test2 = np.array([item[0] for item in test]).reshape(-1,image_size,image_size,3)
y_train = np.array([item[1] for item in train]).reshape(-1)
y_test = np.array([item[1] for item in test]).reshape(-1)

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Create an instance of ImageDataGenerator with rescaling
datagen1 = ImageDataGenerator(rescale=1./255)

# Assuming 'images' is your array of images
# Use the .flow() method to apply the scaling
X_train = datagen1.flow(X_train2)


datagen2 = ImageDataGenerator(rescale=1./255)

# Assuming 'images' is your array of images
# Use the .flow() method to apply the scaling
X_test = datagen2.flow(X_test2)


In [ ]:
model = Sequential([
  layers.Conv2D(8, (4, 4), padding='same', activation='relu', input_shape=(224, 224, 3)),
  layers.MaxPooling2D(pool_size=(2, 2), strides=2),
  layers.Conv2D(16, (4, 4), padding='same', activation='relu'),
  layers.MaxPooling2D(pool_size=(2, 2), strides=2),
  layers.Conv2D(32,  (2, 2), padding='same', activation='relu'),
  layers.Conv2D(64,  (2, 2), padding='same', activation='relu'),
  layers.MaxPooling2D(pool_size=(2, 2), strides=2),
  layers.Conv2D(128,  (2, 2), padding='same', activation='relu'),
  layers.MaxPooling2D(pool_size=(2, 2), strides=2),
  layers.Conv2D(256,  (2, 2), padding='same', activation='relu'),
  layers.MaxPooling2D(pool_size=(2, 2), strides=2),
  layers.Conv2D(256, (2, 2), padding='same', activation='relu'),
  layers.MaxPooling2D(pool_size=(2, 2), strides=2),
  layers.Dropout(0.3),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(1, activation='linear'),




])

In [ ]:
model.compile(loss='mae',optimizer='Adam')
early_stop = EarlyStopping(monitor='val_loss', patience=35, verbose=1)

In [ ]:
epochsize = 300
#model = model.fit(X_train, y_train, validation_split= 0.1,epochs=10, batch_size = 100, callbacks=[early_stop],verbose= 1,shuffle=False)
history = model.fit(X_train, y_train, validation_split= 0.1,epochs=epochsize, batch_size=100, callbacks=[early_stop],verbose= 1,shuffle=False)

In [ ]:
loss = model.evaluate(X_test, y_test)
#accuracy = model.evaluate(X_test, y_test)

In [ ]:
model.summary()

In [ ]:
from sklearn.metrics import mean_absolute_percentage_error as mape

y_pred1 = model.predict(X_test)
y_pred = y_pred1.reshape(-1)
MAE=mae(y_test,y_pred)
RMSE=mse(y_test,y_pred,squared=False)
MAPE=mape(y_test,y_pred)
R2=rSquared(y_test,y_pred)

print("Mean Absolute Error:", MAE)
print("Root Mean Squared Error:", RMSE)
print("Mean Absolute Precentage Error",MAPE)
print("Coefficient of determination ",R2)

In [ ]:
import datetime
x=datetime.datetime.now()
model_name=("{}_{}_{}_{}.hdf5".format(x.month,x.day,x.hour,x.minute))

model.save('/content/drive/MyDrive/FYP/saved_models/saved_CNN_models/'+model_name)

In [ ]:
plt.figure(figsize=(10, 6))

# Plotting y_test values
plt.scatter(y_pred, y_test, c='blue', label='True Values')

max_value = max(max(y_test), max(y_pred))
min_value = min(min(y_test), min(y_pred))
plt.plot([min_value, max_value], [min_value, max_value], color='red',linestyle='dashed',label='Reference line')


plt.xlabel('Predicted PM2.5 values (ug/m3)', fontsize=15)
plt.ylabel('True PM2.5 values (ug/m3)', fontsize=15)
#plt.legend()
#plt.title("Scatter plot of Predicted Vs True PM2.5 Concentrations")
plt.show()